In [1]:
import heapq

class State:
    def __init__(self, lM, lC, boat, rM, rC, parent=None):
        self.lM = lM  # Number of missionaries on the left side
        self.lC = lC  # Number of cannibals on the left side
        self.boat = boat  # Boat location: 0 for left, 1 for right
        self.rM = rM  # Number of missionaries on the right side
        self.rC = rC  # Number of cannibals on the right side
        self.parent = parent  # Parent state (previous state)
        
    def is_valid(self):
        # Check if the state is valid (no missionaries are eaten)
        if self.lM < 0 or self.lC < 0 or self.rM < 0 or self.rC < 0:
            return False
        if self.lM > 0 and self.lM < self.lC:
            return False
        if self.rM > 0 and self.rM < self.rC:
            return False
        return True
    
    def is_goal(self):
        # Check if the state is a goal state (all missionaries and cannibals on the right side)
        return self.lM == 0 and self.lC == 0
    
    def __eq__(self, other):
        return self.lM == other.lM and self.lC == other.lC and \
               self.boat == other.boat and self.rM == other.rM and self.rC == other.rC
    
    def __hash__(self):
        return hash((self.lM, self.lC, self.boat, self.rM, self.rC))
    
    def __lt__(self, other):
        # Compare states based on their total number of people on the left side
        return (self.lM + self.lC) < (other.lM + other.lC)

def get_valid_actions(state):
    actions = []
    if state.boat == 0:  # Boat is on the left side
        for uM in range(3):
            for uC in range(3):
                if 0 < uM + uC <= 2 and state.lM - uM >= 0 and state.lC - uC >= 0:
                    actions.append((uM, uC))
    else:  # Boat is on the right side
        for uM in range(3):
            for uC in range(3):
                if 0 < uM + uC <= 2 and state.rM - uM >= 0 and state.rC - uC >= 0:
                    actions.append((uM, uC))
    return actions

def apply_action(state, action):
    uM, uC = action
    if state.boat == 0:
        return State(state.lM - uM, state.lC - uC, 1, state.rM + uM, state.rC + uC, state)
    else:
        return State(state.lM + uM, state.lC + uC, 0, state.rM - uM, state.rC - uC, state)

def heuristic(state):
    # Heuristic function: estimate the cost to reach the goal state
    # Here, we use a simple heuristic based on the number of people on the left side
    return state.lM + state.lC

def a_star_search():
    initial_state = State(3, 3, 0, 0, 0)
    if initial_state.is_goal():
        return [initial_state]

    frontier = []
    heapq.heappush(frontier, (heuristic(initial_state), initial_state))
    visited = set()

    while frontier:
        _, current_state = heapq.heappop(frontier)

        if current_state.is_goal():
            path = []
            while current_state:
                path.append(current_state)
                current_state = current_state.parent
            path.reverse()
            return path

        visited.add(current_state)

        for action in get_valid_actions(current_state):
            next_state = apply_action(current_state, action)
            if next_state.is_valid() and next_state not in visited:
                heapq.heappush(frontier, (heuristic(next_state), next_state))

    return None

def display_steps(path):
    for i, state in enumerate(path):
        print(f"Step {i}: Boat {'left' if state.boat == 0 else 'right'} side")
        print(f"Left side: {state.lM}M {state.lC}C | Right side: {state.rM}M {state.rC}C")
        print("----")

def main():
    path = a_star_search()
    if path:
        print("Solution found!")
        display_steps(path)
    else:
        print("No solution found.")

if __name__ == "__main__":
    main()


Solution found!
Step 0: Boat left side
Left side: 3M 3C | Right side: 0M 0C
----
Step 1: Boat right side
Left side: 3M 1C | Right side: 0M 2C
----
Step 2: Boat left side
Left side: 3M 2C | Right side: 0M 1C
----
Step 3: Boat right side
Left side: 3M 0C | Right side: 0M 3C
----
Step 4: Boat left side
Left side: 3M 1C | Right side: 0M 2C
----
Step 5: Boat right side
Left side: 1M 1C | Right side: 2M 2C
----
Step 6: Boat left side
Left side: 2M 2C | Right side: 1M 1C
----
Step 7: Boat right side
Left side: 0M 2C | Right side: 3M 1C
----
Step 8: Boat left side
Left side: 0M 3C | Right side: 3M 0C
----
Step 9: Boat right side
Left side: 0M 1C | Right side: 3M 2C
----
Step 10: Boat left side
Left side: 0M 2C | Right side: 3M 1C
----
Step 11: Boat right side
Left side: 0M 0C | Right side: 3M 3C
----
